# Run PPO on Ant from [here](https://github.com/pat-coady/trpo)
#### More descriptions can be found on Patrick Coady's blog regarding [gym and ppo](https://learningai.io/projects/2017/07/28/ai-gym-workout.html) or [descriptions about Ant env](https://gist.github.com/pat-coady/bac60888f011199aad72d2f1e6f5a4fa)

In [ ]:
import tensorflow as tf
from antTrainEnv_class import antTrainEnv_ppo_class
%matplotlib inline  
print ("TF version is [%s]."%(tf.__version__))

In [ ]:
headingCoef = 1e-4 # Heading penalty 
maxSec = 6
maxEpoch  = 10000
batchSize = 100
name = 'ant_ppo_headingCoef%.0e_maxSec%d_batchSize%d'%\
    (headingCoef,maxSec,batchSize)
print ("Name:[%s]"%(name))

In [ ]:
AntPPO = antTrainEnv_ppo_class(_name=name,_headingCoef=headingCoef)

In [ ]:
PLOT_EVERY = 10
AntPPO.train(_maxEpoch=maxEpoch,_batchSize=batchSize,_maxSec=maxSec,_PLOT_EVERY=PLOT_EVERY)